In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense,Flatten,Reshape,Dropout,LeakyReLU,BatchNormalization,Conv2D,Conv2DTranspose
from tensorflow.keras.models import Sequential

In [ ]:
def make_img_grid(imgs, cols, rows):
    axes=[]
    fig=plt.figure()
    num = 0
    for x in range(cols*rows):
        axes.append(fig.add_subplot(rows, cols, x+1) )
        plt.imshow(tf.reshape(imgs[num],[28,28]), cmap="gray")
        plt.axis('off')
        num += 1
    fig.tight_layout()
    plt.show()

In [ ]:
class Model():
    def __init__(self, batch_size, coding_size):
        self.batch_size = batch_size
        self.coding_size = coding_size
        print("Model: initialized")       
    
    def model_info(self):
        self.GAN.summary()
        print("\n###### G ######")
        self.GAN.layers[0].summary()
        print("\n###### D ######")
        self.GAN.layers[1].summary()
    
    def load_model(self, path):
        self.GAN = tf.keras.models.load_model(filepath=path, compile=False)
        self.generator, self.discriminator = self.GAN.layers
        self.discriminator.compile(loss="binary_crossentropy",optimizer="adam")
        self.discriminator.trainable = False
        self.GAN.compile(loss="binary_crossentropy",optimizer="adam")

        self.dataset = tf.data.Dataset.from_tensor_slices(self.data).shuffle(buffer_size=1500)
        self.dataset = self.dataset.batch(self.batch_size,drop_remainder=True).prefetch(1)

    def train_model(self, epochs, path): 
        a = 0
        for epoch in range(epochs):
            print(f"{epoch+1} Epoch von {epochs}")
            b = 0
            if a%15 == 0:
                tf.keras.models.save_model(self.GAN, path)
            a += 1
            
            for batch in self.dataset:
                b += 1
                if b%50 == 0:
                    print(f"\t{b} Batch von {len(self.data)//self.batch_size}")

                #Diskriminatortraining
                Z = tf.random.normal(shape=[self.batch_size, self.coding_size])
                fake_imgs = self.generator(Z)
                self.discriminator.trainable = True
                self.discriminator.train_on_batch(tf.concat([fake_imgs, tf.dtypes.cast(batch,tf.float32)],axis=0), 
                                                  tf.constant([[0.0]] * self.batch_size + [[1.0]] * self.batch_size))
                
                #Generatortraining
                self.discriminator.trainable = False
                self.GAN.train_on_batch(tf.random.normal(shape=[self.batch_size, self.coding_size]),
                                        tf.constant([[1.0]] * self.batch_size))

        tf.keras.models.save_model(self.GAN, path)
    
    def generate_images(self, size, show=True):
        Z = tf.random.normal(shape=[size,self.coding_size])
        images = self.generator(Z)
        if show:
          for image in images:
              plt.imshow(image.numpy().reshape(28,28),cmap="gray")
              plt.show()
        return images

In [ ]:
class GAN(Model):
    def prepare_data(self, number="ALL"):
        (X_train, y_train) , (X_test, y_test) = mnist.load_data()
        if number == "ALL":
          self.data = X_train
        else:
          self.data = X_train[y_train==number]   

    def prepare_GAN(self):
        #Diskriminator      ==> 28x28=784 zu 150 zu 100 zu 1 
        discriminator = Sequential()
        discriminator.add(Flatten(input_shape=[28,28]))
        discriminator.add(Dense(150,activation="relu"))
        discriminator.add(Dense(100,activation="relu"))
        discriminator.add(Dense(1,activation="sigmoid"))

        #Generator          ==> 100 zu 150 zu 784 zu 28x28
        generator = Sequential() 
        generator.add(Dense(100,activation="relu",input_shape=[self.coding_size]))
        generator.add(Dense(150,activation="relu"))
        generator.add(Dense(784,activation="sigmoid"))
        generator.add(Reshape([28,28]))

        self.GAN = Sequential([generator,discriminator])
        discriminator.compile(loss="binary_crossentropy",optimizer="adam")
        discriminator.trainable = False
        self.GAN.compile(loss="binary_crossentropy", optimizer="adam")

        self.dataset = tf.data.Dataset.from_tensor_slices(self.data).shuffle(buffer_size=1500)
        self.dataset = self.dataset.batch(self.batch_size,drop_remainder=True).prefetch(1)
        self.generator, self.discriminator = self.GAN.layers

In [ ]:
class DCGAN(Model):
    def prepare_data(self, number="ALL"):
        (X_train, y_train) , (X_test, y_test) = mnist.load_data()
        X_train = X_train/255 #==> Normalisiert Werte auf Bereich zwischen 0 und 1 
        X_train = X_train.reshape(-1,28,28,1) * 2 - 1 #==> Werte zwischen -1 und 1, da tanh in letzter Generator Layer
        if number == "ALL":
          self.data = X_train
        else:
          self.data = X_train[y_train==number]   

    def prepare_DCGAN(self):
        #Diskriminator
        discriminator = Sequential()
        discriminator.add(Conv2D(64,kernel_size=5,strides=2,padding="same",activation=LeakyReLU(0.2),input_shape=[28,28,1])) #==> 28x28x1 zu 14x14x64
        discriminator.add(Dropout(0.4))
        discriminator.add(Conv2D(128,kernel_size=5,strides=2,padding="same",activation=LeakyReLU(0.2))) #==> 14x14x64 zu 7x7x128
        discriminator.add(Dropout(0.4))
        discriminator.add(Flatten()) #==> 7x7x128 zu 6272
        discriminator.add(Dense(1,activation="sigmoid")) #==> O oder 1
        
        #Generator
        generator = Sequential()
        generator.add(Dense(7*7*128,input_shape=[self.coding_size])) #==> 6272
        generator.add(Reshape([7,7,128])) #==> 6272 zu 7x7x128
        generator.add(BatchNormalization())
        generator.add(Conv2DTranspose(64,kernel_size=5,strides=2, padding="same",activation="relu")) #==> 7x7x128 zu 14x14x64
        generator.add(BatchNormalization())
        generator.add(Conv2DTranspose(1,kernel_size=5,strides=2, padding="same",activation="tanh")) #==> 14x14x64 zu 28x28x1

        self.GAN = Sequential([generator,discriminator])
        discriminator.compile(loss="binary_crossentropy",optimizer="adam")
        discriminator.trainable = False
        self.GAN.compile(loss="binary_crossentropy",optimizer="adam")

        self.dataset = tf.data.Dataset.from_tensor_slices(self.data).shuffle(buffer_size=1500)
        self.dataset = self.dataset.batch(self.batch_size,drop_remainder=True).prefetch(1)
        self.generator, self.discriminator = self.GAN.layers

In [ ]:
gan = GAN(batch_size=32,coding_size=125)
gan.prepare_data(number=0)
gan.prepare_GAN()
gan.train_model(epochs=1,path="SavedModelsGAN.h5")

In [ ]:
make_img_grid(gan.generate_images(size=8,show=False), 4, 2)

In [ ]:
dcgan = DCGAN(batch_size=30,coding_size=125)
dcgan.prepare_data(number=0)
dcgan.prepare_DCGAN()
dcgan.train_model(epochs=1,path="SavedModelsDCGAN.h5")

In [ ]:
make_img_grid(dcgan.generate_images(size=8,show=False), 4, 2)